**Используя RandomForestClassifier и еще 2 метода МО предсказать будущий ABC класс на основе любых фичей**

In [13]:
import pandas as pd
import numpy as np


In [23]:
path = '/content/DataSet_w_NA (1).xlsx'
df_init = pd.read_excel(path, sheet_name="Испорченные факты")
df = df_init
df

,Номер заказа,Возврат,Дата заказа,Дата отправки,Режим отправки,Факты.Контрагент ID,Факты.Товар ID,"Продажи, руб","Продажи, шт",Скидка,Маржинальная прибыль,Повторение заказа,Повторение контрагента,Повторение товара
0,2014-100279,Нет,2015-09-10 00:00:00,2015-09-14 00:00:00,Стандарт,CLNT0000422,PROD0000870,11190.0,2.0,0.0,5371.20,1,7,3
1,2014-100293,Нет,2015-09-14 00:00:00,2015-09-18 00:00:00,Стандарт,CLNT0000784,PROD0000646,45528.0,6.0,0.2,15934.80,1,4,8
2,2014-100328,Нет,2015-07-28 00:00:00,2015-08-03 00:00:00,Стандарт,CLNT0000342,PROD0000187,1964.0,1.0,0.2,662.85,1,4,12
3,2014-100363,Нет,2015-10-08 00:00:00,2015-10-15 00:00:00,Стандарт,CLNT0000146,PROD0001095,1184.0,2.0,0.2,414.40,2,12,7
4,2014-100363,Нет,2015-10-08 00:00:00,2015-10-15 00:00:00,Стандарт,CLNT0000146,PROD0001616,9504.0,3.0,0.2,3445.20,2,12,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10288,2018-169929,Нет,2019-03-25 00:00:00,2019-03-28 00:00:00,Первый класс,CLNT0000182,PROD0000869,19950.0,5.0,0.0,5187.00,1,14,6
10289,2018-169978,Нет,2019-06-24 00:00:00,2019-06-28 00:00:00,Стандарт,CLNT0000575,PROD0000293,135882.0,2.0,0.1,24156.80,2,11,11
10290,2018-169978,Нет,2019-06-24 00:00:00,2019-06-28 00:00:00,Стандарт,CLNT0000575,PROD0001131,7188.0,3.0,0.2,2425.95,2,11,4
10291,2018-169999,Нет,2018-12-19 00:00:00,2018-12-26 00:00:00,Стандарт,CLNT0000013,PROD0000549,5784.0,3.0,0.2,1879.80,2,11,13


In [24]:
df = pd.pivot_table(df, values=['Продажи, руб', 'Продажи, шт','Повторение заказа', 'Маржинальная прибыль','Повторение товара'],
                    index=["Факты.Товар ID"], aggfunc={'Продажи, шт': [np.median, np.sum], 'Продажи, руб': np.sum, 
                                                       'Повторение заказа': np.sum, 'Маржинальная прибыль': np.sum})

In [25]:
df.head()

Маржинальная прибыль Повторение заказа Продажи, руб  \
                                sum               sum          sum   
Факты.Товар ID                                                       
PROD0000001               -28160.70                 6     631978.5   
PROD0000002               963721.00                32    5318764.0   
PROD0000003                32200.55                18      76755.0   
PROD0000004              -266380.65                17    3621384.0   
PROD0000005                31874.40                18     150984.0   

               Продажи, шт        
                    median   sum  
Факты.Товар ID                    
PROD0000001            2.5  12.0  
PROD0000002            3.5  51.0  
PROD0000003            2.0  23.0  
PROD0000004            4.0  29.0  
PROD0000005            5.0  24.0

In [26]:
newname=df.columns.map('_'.join)
newname

Index(['Маржинальная прибыль_sum', 'Повторение заказа_sum', 'Продажи, руб_sum',
       'Продажи, шт_median', 'Продажи, шт_sum'],
      dtype='object')

In [27]:
df.columns=newname
df.head()

,Маржинальная прибыль_sum,Повторение заказа_sum,"Продажи, руб_sum","Продажи, шт_median","Продажи, шт_sum"
Факты.Товар ID,,,,,
PROD0000001,-28160.70,6,631978.5,2.5,12.0
PROD0000002,963721.00,32,5318764.0,3.5,51.0
PROD0000003,32200.55,18,76755.0,2.0,23.0
PROD0000004,-266380.65,17,3621384.0,4.0,29.0
PROD0000005,31874.40,18,150984.0,5.0,24.0


In [28]:
df['Продажи, шт_sum']

Факты.Товар ID
PROD0000001    12.0
PROD0000002    51.0
PROD0000003    23.0
PROD0000004    29.0
PROD0000005    24.0
               ... 
PROD0002385    12.0
PROD0002386    12.0
PROD0002388    13.0
PROD0002389     5.0
PROD0002391    12.0
Name: Продажи, шт_sum, Length: 2064, dtype: float64

In [30]:
total_sale=df['Продажи, руб_sum'].sum()
df['Доля']=df['Продажи, руб_sum']/total_sale * 100
df = df.assign(sum_d=df['Доля'].cumsum())

In [32]:
df.loc[(df['sum_d'] <= 80), 'ABC'] = 'A'
df.loc[(df['sum_d'] > 80) & (df['sum_d'] <= 95), 'ABC'] = 'B'
df.loc[(df['sum_d'] > 95), 'ABC'] = 'C'
df

,Маржинальная прибыль_sum,Повторение заказа_sum,"Продажи, руб_sum","Продажи, шт_median","Продажи, шт_sum",Доля,sum_d,ABC
Факты.Товар ID,,,,,,,,
PROD0000001,-28160.70,6,631978.5,2.5,12.0,0.053130,0.053130,A
PROD0000002,963721.00,32,5318764.0,3.5,51.0,0.447148,0.500278,A
PROD0000003,32200.55,18,76755.0,2.0,23.0,0.006453,0.506731,A
PROD0000004,-266380.65,17,3621384.0,4.0,29.0,0.304449,0.811180,A
PROD0000005,31874.40,18,150984.0,5.0,24.0,0.012693,0.823873,A
...,...,...,...,...,...,...,...,...
PROD0002385,572555.90,1,226677.8,12.0,12.0,0.019057,99.932396,C
PROD0002386,2941491.10,1,0.0,12.0,12.0,0.000000,99.932396,C
PROD0002388,557750.00,1,0.0,13.0,13.0,0.000000,99.932396,C


In [33]:
piv_df = pd.DataFrame({"A": ["foo", "foo", "foo", "foo", "foo",
                         "bar", "bar", "bar", "bar"],
                   "B": ["one", "one", "one", "two", "two",
                         "one", "one", "two", "two"],
                   "C": ["small", "large", "large", "small",
                         "small", "large", "small", "small",
                         "large"],
                   "D": [1, 2, 2, 3, 3, 4, 5, 6, 7],
                   "E": [2, 4, 5, 5, 6, 6, 8, 9, 9]})

In [37]:
df['Стоимость, руб'] = df['Продажи, руб_sum']/df['Продажи, шт_sum']
df['Продажи в следующем периоде']= (df['Продажи, шт_sum'] + df['Продажи, шт_median']) *df['Стоимость, руб']
total_sale_next=df['Продажи в следующем периоде'].sum()
total_sale_next

1448727365.7537313

In [38]:
df['Доля_будущая']=df['Продажи в следующем периоде']/total_sale_next * 100
df = df.sort_values(by=('Продажи в следующем периоде'), ascending=False)
df=df.assign(sum_d_next=df['Доля_будущая'].cumsum())
df['Доля_будущая'].sum()

100.00000000000001

In [39]:
df.loc[(df['sum_d_next'] <= 80), 'ABC_next'] = 'A'
df.loc[(df['sum_d_next'] > 80) & (df['sum_d_next'] <= 95), 'ABC_next'] = 'B'
df.loc[(df['sum_d_next'] > 95), 'ABC_next'] = 'C'
df.loc[(df['ABC'] != df['ABC_next']), 'Изменение класса'] = 1
df.loc[(df['ABC'] == df['ABC_next']), 'Изменение класса'] = 0
df['Изменение класса'].sum()

1449.0

In [40]:
df.loc[(df['sum_d'] <= 80), 'class'] = '0'
df.loc[(df['sum_d'] > 80) & (df['sum_d'] <= 95), 'class'] = '1'
df.loc[(df['sum_d'] > 95), 'class'] = '2'
df.loc[(df['sum_d_next'] <= 80), 'class_next'] = '0'
df.loc[(df['sum_d_next'] > 80) & (df['sum_d_next'] <= 95), 'class_next'] = '1'
df.loc[(df['sum_d_next'] > 95), 'class_next'] = '2'
df

,Маржинальная прибыль_sum,Повторение заказа_sum,"Продажи, руб_sum","Продажи, шт_median","Продажи, шт_sum",Доля,sum_d,ABC,"Стоимость, руб",Продажи в следующем периоде,Доля_будущая,sum_d_next,ABC_next,Изменение класса,class,class_next
Факты.Товар ID,,,,,,,,,,,,,,,,
PROD0001406,1.259996e+07,17,30799912.0,4.0,20.0,2.589344,79.059165,A,1.539996e+06,3.695989e+07,2.551197,2.551197,A,0.0,0,0
PROD0001430,-9.055392e+05,7,11319240.0,6.0,6.0,0.951607,80.734508,B,1.886540e+06,2.263848e+07,1.562646,4.113843,A,1.0,1,0
PROD0000454,3.876520e+06,23,13726692.0,3.0,31.0,1.154001,27.730567,A,4.427965e+05,1.505508e+07,1.039194,5.153037,A,0.0,0,0
PROD0000488,1.746230e-10,15,10935288.0,5.0,39.0,0.919328,31.449419,A,2.803920e+05,1.233725e+07,0.851592,6.004629,A,0.0,0,0
PROD0001657,2.047488e+06,6,9187447.5,4.0,12.0,0.772387,89.712642,B,7.656206e+05,1.224993e+07,0.845565,6.850194,A,1.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PROD0002137,1.623771e+06,1,0.0,9.0,9.0,0.000000,98.549333,C,0.000000e+00,0.000000e+00,0.000000,100.000000,C,0.0,2,2
PROD0002140,4.565262e+06,2,0.0,11.5,23.0,0.000000,98.549333,C,0.000000e+00,0.000000e+00,0.000000,100.000000,C,0.0,2,2
PROD0002024,2.473960e+04,1,0.0,4.0,4.0,0.000000,97.417387,C,0.000000e+00,0.000000e+00,0.000000,100.000000,C,0.0,2,2
